In [9]:
# https://arize.com/blog-course/react-agent-llm/

In [12]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
# from langchain.llms import OpenAI

from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_community.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "..." # https://platform.openai.com/

In [10]:
# !pip install langchain-experimental

In [4]:
# llm = OpenAI(temperature=0) #Temp = 0 generally means less hallucinations and a more deterministic model
# https://platform.openai.com/docs/models/model-endpoint-compatibility
# llm = OpenAI(model="gpt-3.5-turbo-instruct") # /v1/completions (Legacy)
# llm = OpenAI(model="gpt-4-1106-preview", temperature=0)
# llm = OpenAI(model="gpt-3.5-turbo-0301", temperature=0)
# Choose the LLM that will drive the agent
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-1106")

#Tools are functions that agents can use to interact with the world. These tools can be generic utilities (e.g. search), other chains, or even other agents.
tools = load_tools(["terminal"], llm=llm) #The tools the agent will have access to are: [Google search, math, wikipedia, terminal]

In [5]:
llm.model_name

'gpt-3.5-turbo-1106'

In [20]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/structured-chat-agent")
print(prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and s

In [39]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain

prefix = """Assist user the best you can. You have access to the following optional tools:"""
suffix = """Begin! Remember that you can use  run for calling the terminal.

Question: {input}
{agent_scratchpad}
{tool_names}
{tools}"""

prompt = ZeroShotAgent.create_prompt(
   tools, prefix=prefix, suffix=suffix, input_variables=["input", "agent_scratchpad", 'tool_names', 'tools']
)
print(prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] template='Assist user the best you can. You have access to the following optional tools:\n\nterminal: Run shell commands on this Linux machine.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [terminal]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin! Remember that you can use  run for calling the terminal.\n\nQuestion: {input}\n{agent_scratchpad}\n{tool_names}\n{tools}'


In [14]:
# zero-shot-react-description
"""agent = initialize_agent(tools,
                         llm,
                         agent="zero-shot-react-description",
                         verbose=True)"""

# conversational-react-description
"""agent = initialize_agent(tools,
                         llm,
                         agent="conversational-react-description",
                         verbose=True)"""



In [40]:
tools[0].name, tools[0].description

('terminal', 'Run shell commands on this Linux machine.')

In [41]:
import warnings

# Suppress UserWarnings
warnings.filterwarnings('ignore', category=UserWarning)

In [42]:
# Chat
agent = create_structured_chat_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [43]:
agent_executor.invoke({"input": "Check the date and the list of the current directory"})



> Entering new AgentExecutor chain...
Could not parse LLM output: Action Input: date
Invalid or incomplete responseCould not parse LLM output: Action Input: ls

Invalid or incomplete responseCould not parse LLM output: I apologize for the confusion. It seems there was an issue with retrieving the date and the list of the current directory. Let me try running the commands in the terminal to get the information.

Action: terminal
Action Input: ['date', 'ls']

Invalid or incomplete responseCould not parse LLM output: Final Answer: I apologize, but I was unable to retrieve the date and the list of the current directory.Invalid or incomplete responseCould not parse LLM output: I apologize, but it seems there is an issue with retrieving the date and the list of the current directory. I will need to troubleshoot this further.Invalid or incomplete responseCould not parse LLM output: I apologize, but it seems there is an issue with retrieving the date and the list of the current directory. I 

{'input': 'Check the date and the list of the current directory',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [37]:
agent_executor.run("Check the date and the list of the current directory")

ValueError: `run` not supported when there is not exactly one output key. Got [].

In [50]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)


tool_names = [tool.name for tool in tools]


agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)


agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)


agent_executor.run("Check the date and the list of the current directory")



> Entering new AgentExecutor chain...
Thought: I should use the terminal to run the 'date' and 'ls' commands
Action: terminal
Action Input: date
Observation: Sat Jan  6 04:33:23 PM +04 2024

Thought: I should also check the current directory
Action: terminal
Action Input: ls
Observation: app.py
Dockerfile
react.ipynb
requirements.txt
terminal_fake.ipynb
terminal_native.ipynb
tools.ipynb

Thought: I now know the final answer
Final Answer: The date is Sat Jan  6 04:33:23 PM +04 2024 and the current directory contains the files app.py, Dockerfile, react.ipynb, requirements.txt, terminal_fake.ipynb, terminal_native.ipynb, and tools.ipynb.

> Finished chain.


'The date is Sat Jan  6 04:33:23 PM +04 2024 and the current directory contains the files app.py, Dockerfile, react.ipynb, requirements.txt, terminal_fake.ipynb, terminal_native.ipynb, and tools.ipynb.'

In [51]:
agent_executor.run("My name is Bob. What is your name?")



> Entering new AgentExecutor chain...
Thought: I should introduce myself and ask for the user's name.
Action: terminal
Action Input: whoami
Observation: alex

Thought: I should greet the user and ask for their name.
Action: terminal
Action Input: echo "Hello, what is your name?"
Observation: /bin/sh: 1: Syntax error: Unterminated quoted string

Thought: I should use the read command to get the user's input.
Action: terminal
Action Input: read name
Observation: 
Thought: I should greet the user by their name.
Action: terminal
Action Input: echo "Hello $name, my name is Alex."
Observation: /bin/sh: 1: Syntax error: Unterminated quoted string

Thought: I should ask the user how I can assist them.
Action: terminal
Action Input: echo "How can I assist you?"
Observation: /bin/sh: 1: Syntax error: Unterminated quoted string

Thought: I should wait for the user's input.
Action: terminal
Action Input: read question
Observation: 
Thought: I should provide an answer based on the user's input.
A

"I'm sorry, I am just a simple AI and I am not able to assist with that question."

In [52]:
agent_executor.run("Do you remember my name? Call my name please")



> Entering new AgentExecutor chain...
Thought: I should check if I have access to the user's name
Action: terminal
Action Input: whoami
Observation: alex

Thought: I can use the user's name to call them
Action: terminal
Action Input: echo "Hello, alex!"
Observation: /bin/sh: 1: Syntax error: Unterminated quoted string

Thought: I need to use double quotes to properly format the string
Action: terminal
Action Input: echo "Hello, alex!"
Observation: /bin/sh: 1: Syntax error: Unterminated quoted string

Thought: I should try using single quotes instead
Action: terminal
Action Input: echo 'Hello, alex!'
Observation: Hello, alex!

Thought: I have successfully called the user by their name
Final Answer: Hello, alex!

> Finished chain.


'Hello, alex!'